# Peter's Clustering Cities in Toronto 

<b> Code to import libraries </b>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [11]:
from bs4 import BeautifulSoup
import requests

In [18]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [19]:
page = requests.get(url)
page

<Response [200]>

In [21]:
soup = BeautifulSoup(page.content, 'html.parser')

In [171]:
rows = soup.find_all('tr')
rowlength = len(rows) - 5 #because last few rows are styling

In [166]:
### define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [167]:
for index in range (1, rowlength):
    PostalCode = rows[index].find_all('td')[0].get_text()
    Borough = rows[index].find_all('td')[1].get_text()
    Neighborhood = rows[index].find_all('td')[2].get_text().splitlines()[0]
    str(Neighborhood)
    
    neighborhoods = neighborhoods.append({'PostalCode': PostalCode,
                                          'Borough': Borough,
                                          'Neighborhood': Neighborhood}, ignore_index=True)




### 3. Cleaning up Data to drop row if borough is not assigned, make neighborhood borough if no neighborhood

In [180]:
#Drop Row if Borough is null
neighborhoods['Borough'] = neighborhoods['Borough'].replace('Not assigned', np.nan)
neighborhoods = neighborhoods.dropna(axis=0, subset=['Borough'])

In [184]:
#Change neighborhood to borough if neighborhood = not assigned
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].replace('Not assigned', neighborhoods['Borough'])

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#### Now I group by PostalCode and Borough and append the neighborhoods together

In [241]:
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].astype(str)

In [265]:
result = neighborhoods.groupby(['PostalCode', 'Borough']).agg(', '.join).reset_index()

In [266]:
result

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [267]:
result.shape

(103, 3)

# Part 2, Add Lat and Long to Dataframe

In [248]:
!wget -q -O 'data.csv' http://cocl.us/Geospatial_data
print('Data Downloaded!')

Data Downloaded!


In [308]:
data = pd.read_csv('data.csv')

In [288]:
#Rename Postal Code to PostalCode in data so you can merge
#data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [309]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [310]:
#toronto = pd.merge(result, 
 #                  data[['Latitude', 'Longitude']],
  #                on='PostalCode')

toronto = pd.concat([result,data], axis=1)

In [316]:
toronto.drop(columns=['Postal Code'], inplace=True)

In [317]:
toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Part 3 Clustering

In [ ]:
#Install and import all libraries
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [320]:
#Downtown Canada Location received from table above, CSV provided by Coursera
latitude = 43.648429
longitude = -79.382280


In [324]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Cluster neighborhoods in Toronto

In [333]:
#onehot encoding
toronto_grouped = toronto.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Latitude,Longitude
0,"Adelaide, King, Richmond",43.650571,-79.384568
1,Agincourt,43.794200,-79.262029
2,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
4,"Alderwood, Long Branch",43.602414,-79.543484
5,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
6,Bayview Village,43.786947,-79.385975
7,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
8,Berczy Park,43.644771,-79.373306
9,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Run K means cluster

In [336]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 3, 3, 0, 0, 4, 4, 4, 2, 1], dtype=int32)

In [ ]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)


In [343]:
toronto_grouped

,Cluster Labels,Neighborhood,Latitude,Longitude
0,2,"Adelaide, King, Richmond",43.650571,-79.384568
1,3,Agincourt,43.794200,-79.262029
2,3,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
3,0,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
4,0,"Alderwood, Long Branch",43.602414,-79.543484
5,4,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
6,4,Bayview Village,43.786947,-79.385975
7,4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
8,2,Berczy Park,43.644771,-79.373306
9,1,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Put it on a map!

In [344]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_grouped['Latitude'], toronto_grouped['Longitude'], toronto_grouped['Neighborhood'], toronto_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters